In [1]:
import pandas as pd

# 휴장일 엑셀 파일 로드
hdays_2021 = pd.read_excel('./excel/2021_hdays.xls')

hdays_2021.head()

,일자 및 요일,요일구분,비고
0,2021-01-01,금요일,신정
1,2021-02-11,목요일,설날
2,2021-02-12,금요일,설날
3,2021-03-01,월요일,삼일절
4,2021-05-05,수요일,어린이날


In [2]:
# 불필요 컬럼 제거
hdays_2021.drop(columns=["요일구분", "비고"], inplace=True)

hdays_2021.head()

,일자 및 요일
0,2021-01-01
1,2021-02-11
2,2021-02-12
3,2021-03-01
4,2021-05-05


In [3]:
# 불필요 문자 제거
hdays_2021["일자 및 요일"] = hdays_2021["일자 및 요일"].str.replace("-","")
# int로 데이터타입 변환
hdays_2021["일자 및 요일"] = hdays_2021["일자 및 요일"].astype(int)

In [4]:
# 휴장일 리스트 생성
hdays_list = hdays_2021["일자 및 요일"].values.tolist()

hdays_list

[20210101,
 20210211,
 20210212,
 20210301,
 20210505,
 20210519,
 20210816,
 20210920,
 20210921,
 20210922,
 20211004,
 20211011,
 20211231]

In [5]:
# 2019-01-01부터 2019-12-31 까지 영업일 리스트 생성
bdays = pd.date_range('2021-01-01', '2021-12-31', freq='B')
df = pd.DataFrame()
df['bdays'] = bdays
df['bdays'] = df['bdays'].dt.strftime('%Y%m%d') 
df['bdays'] = df['bdays'].astype(int)

bdays_list = df['bdays'].tolist()
bdays_list

[20210101,
 20210104,
 20210105,
 20210106,
 20210107,
 20210108,
 20210111,
 20210112,
 20210113,
 20210114,
 20210115,
 20210118,
 20210119,
 20210120,
 20210121,
 20210122,
 20210125,
 20210126,
 20210127,
 20210128,
 20210129,
 20210201,
 20210202,
 20210203,
 20210204,
 20210205,
 20210208,
 20210209,
 20210210,
 20210211,
 20210212,
 20210215,
 20210216,
 20210217,
 20210218,
 20210219,
 20210222,
 20210223,
 20210224,
 20210225,
 20210226,
 20210301,
 20210302,
 20210303,
 20210304,
 20210305,
 20210308,
 20210309,
 20210310,
 20210311,
 20210312,
 20210315,
 20210316,
 20210317,
 20210318,
 20210319,
 20210322,
 20210323,
 20210324,
 20210325,
 20210326,
 20210329,
 20210330,
 20210331,
 20210401,
 20210402,
 20210405,
 20210406,
 20210407,
 20210408,
 20210409,
 20210412,
 20210413,
 20210414,
 20210415,
 20210416,
 20210419,
 20210420,
 20210421,
 20210422,
 20210423,
 20210426,
 20210427,
 20210428,
 20210429,
 20210430,
 20210503,
 20210504,
 20210505,
 20210506,
 20210507,

In [6]:
odays = [x for x in bdays_list if x not in hdays_list]
len(odays)

248

In [9]:
# 구글 크롬 사용을 위해 webdriver 따로 다운로드 후 불러오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# chromedirver 불러오기
driver = webdriver.Chrome('./chromedriver.exe')
driver.maximize_window()
time.sleep(1)

# krx 링크 열기
krxStatUrl = "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201"
driver.get(krxStatUrl)

# XPath를 사용한 좌측 탭의 "기본통계" - "지수" - "주가지수" - "지수구성종목" 클릭
krxIndex = driver.find_element(By.XPATH, 
                               "//*[@id='jsMdiMenu']/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[1]/a")
# "지수" 클릭
driver.execute_script("arguments[0].click();", krxIndex)

# "주가지수" XPath
stockIndex = driver.find_element(By.XPATH, 
                               "//*[@id='jsMdiMenu']/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[1]/ul/li[1]/a")
# "주가지수" 클릭
driver.execute_script("arguments[0].click();", stockIndex)

# "지수구성종목" XPath
indexList = driver.find_element(By.XPATH, 
                               "//*[@id='jsMdiMenu']/div[4]/ul/li[1]/ul/li[2]/div/div[1]/ul/li[1]/ul/li[1]/ul/li[6]/a")
# "지수구성종목" 클릭
driver.execute_script("arguments[0].click();", indexList)

# 지수명 input XPath
indexSearch = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH,
                                            "/html/body/div[2]/section[2]/section/section/div/div/form/div[1]/div/table/tbody/tr[1]/td/div/div/p/input")))
indexSearch.click()
indexSearch.send_keys("코스피 200")
indexSearch.send_keys(Keys.ENTER)

# 코스피 200 버튼
kospi200Btn = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, 
                                           "//*[@id='jsGrid__finder_equidx0_0']/tbody/tr[1]")))
kospi200Btn.click()

# 모든 날짜에서 휴장일과 주말을 뺀 개장일 리스트
odays = [x for x in bdays_list if x not in hdays_list]

for oday in odays:
    # 조회일자 id
    DateSearch = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.ID,
                                                      "trdDd")))
    
    DateSearch.click()
    DateSearch.send_keys(Keys.CONTROL + "a")
    DateSearch.send_keys(Keys.BACKSPACE)
    DateSearch.send_keys(oday)

    # "조회" 버튼 XPath
    SearchBtn = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH,
                                                "//*[@id='jsSearchButton']")))
    SearchBtn.click()
    
    # 다운로드 버튼 class
    downloadBtn = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 
                                               "CI-MDI-UNIT-DOWNLOAD")))
    downloadBtn.click()
    
    # csv 다운로드 버튼 XPath
    csvBtn = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.LINK_TEXT, 
                                               "CSV")))
    csvBtn.click()

C:\Users\wndgj\AppData\Local\Temp\ipykernel_21752\2435432713.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x00936463+2188387]
	Ordinal0 [0x008CE461+1762401]
	Ordinal0 [0x007E3D78+802168]
	Ordinal0 [0x00811880+989312]
	Ordinal0 [0x00811B1B+989979]
	Ordinal0 [0x0083E912+1173778]
	Ordinal0 [0x0082C824+1099812]
	Ordinal0 [0x0083CC22+1166370]
	Ordinal0 [0x0082C5F6+1099254]
	Ordinal0 [0x00806BE0+945120]
	Ordinal0 [0x00807AD6+948950]
	GetHandleVerifier [0x00BD71F2+2712546]
	GetHandleVerifier [0x00BC886D+2652765]
	GetHandleVerifier [0x009C002A+520730]
	GetHandleVerifier [0x009BEE06+516086]
	Ordinal0 [0x008D468B+1787531]
	Ordinal0 [0x008D8E88+1805960]
	Ordinal0 [0x008D8F75+1806197]
	Ordinal0 [0x008E1DF1+1842673]
	BaseThreadInitThunk [0x75A3FA29+25]
	RtlGetAppContainerNamedObjectPath [0x777A7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x777A7A6E+238]
